In [1]:
import numpy as np
import mysql.connector
import pandas as pd
import plotly.graph_objects as go
import os
import datetime
import pytz
import googlemaps

In [3]:
cnx = mysql.connector.connect(user='admin', password='actuadores',
                              host='157.230.209.3',
                              database='monitoreodb')

query = "SELECT * from operation WHERE vehicle_id = 'GHW284'"
GHW284 = pd.read_sql_query(query, cnx, index_col='id')
GHW284.dropna(axis=1, how='all', inplace=True)

In [24]:
ruta_1 = GHW284[(GHW284.timestamp > '2020-12-11 14:19') &
                  (GHW284.timestamp < '2020-12-11 18:16')]
ruta_1['Slope (°)'] = abs(ruta_1['slope']) + 0.2

<ipython-input-24-dcf10868a870>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [53]:
fig = go.Figure(
    go.Scattermapbox(
        mode="markers+lines",
        lon=ruta_1["longitude"],
        lat=ruta_1["latitude"],
        marker={"size": 5, "color":ruta_1["Slope (°)"]},
    )
)
fig.add_trace(
    go.Scattermapbox(
        mode="markers+lines",
        lon=ruta_1["longitude"].iloc[0:2],
        lat=ruta_1["latitude"].iloc[0:2],
        marker={"size": 12, "color":"red"},
    )
)

fig.update_layout(
    margin={"l": 0, "t": 0, "b": 0, "r": 0},
    mapbox={
        "center": {"lon": -75.58, "lat": 6.151},
        "style": "stamen-terrain",
        "zoom": 10,
    },
)

In [41]:
import plotly.express as px
ini = ruta_1.iloc[0:2]
fin = ruta_1.iloc[-2:-1]

px.set_mapbox_access_token("pk.eyJ1Ijoic2FudGlhZ28xNzFjYyIsImEiOiJja2NjZTkzOTkwM3ZxMndxa296YTVseGNkIn0.EFcvvL83cLQZYsqSgLVa6A")
fig = px.scatter_mapbox(ruta_1, lat="latitude", lon="longitude",     color="Slope (°)", 
                  color_continuous_scale=px.colors.sequential.Plasma, size_max=15, zoom=10)
# fig.add_trace(px.scatter_mapbox(ini, lat="latitude", lon="longitude"))
fig.show()